In [1]:
import itertools
import random

import IPython.display
from matplotlib.axes import Axes
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# df_orig = pd.read_csv('../logs/rundirs/sorted-20241116_215630.csv')
# df_orig = pd.read_csv('../logs/rundirs/sorted-20241119_223637.csv')
# df_orig = pd.read_csv('../logs/rundirs/sorted-20241120_143312.csv')
df_orig = pd.read_csv('../logs/rundirs/sorted-20241120_175210.csv')
df_orig

,Date,Scenario ID,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of critical sections,No. of near-misses,No. of collisions,Is blocked,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s)
0,20241120_155651,map-generator/generated-maps/2024-11-20_14:24:...,0,HumanDrivenVehicle,78.03,0,62.2,2,0,146,0,0,1,567.1,547.2,599.9,0.3,5.6,0.1
1,20241120_155845,map-generator/generated-maps/2024-11-20_14:24:...,0,HumanDrivenVehicle,78.03,0,61.8,1,1,146,0,0,1,574.4,573.3,599.9,0.3,5.6,0.1
2,20241120_160036,map-generator/generated-maps/2024-11-20_14:24:...,0,HumanDrivenVehicle,78.03,0,61.8,1,1,146,0,0,1,574.4,573.3,599.9,0.3,5.6,0.1
3,20241120_160224,map-generator/generated-maps/2024-11-20_14:24:...,0,HumanDrivenVehicle,78.20,1,80.3,33,0,166,0,2,0,536.0,29.7,599.9,0.3,5.6,0.1
4,20241120_160424,map-generator/generated-maps/2024-11-20_14:24:...,0,HumanDrivenVehicle,78.20,12,998.7,25,21,159,1,6,0,71.9,16.2,599.9,0.3,5.6,1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,20241120_174300,map-generator/generated-maps/2024-11-20_14:24:...,4,AutonomousVehicle,62.11,13,1602.3,18,0,79,0,0,0,51.8,10.2,599.9,0.3,5.6,2.7
296,20241120_174500,map-generator/generated-maps/2024-11-20_14:24:...,4,AutonomousVehicle,62.11,1,180.6,4,0,16,0,0,1,520.3,496.4,599.9,0.3,5.6,0.3
297,20241120_174623,map-generator/generated-maps/2024-11-20_14:24:...,4,AutonomousVehicle,60.61,15,1849.8,16,0,55,0,3,0,12.6,1.4,599.9,0.3,5.6,3.1
298,20241120_174826,map-generator/generated-maps/2024-11-20_14:24:...,4,AutonomousVehicle,60.61,12,1459.5,18,0,74,0,0,0,63.9,13.7,599.9,0.3,5.6,2.4


In [3]:
df_id = df_orig['Scenario ID'].str.split(r'[;,] ', expand=True)
df_id.columns = ['filename', 'Coordination strategy', 'string_seed', 'string_probabilityForcingForHuman', 'heuristic']
df_id = pd.concat([
    df_id,
    df_id['filename'].str.extract(r'(?P<dir_map>[^/]+)/[^/]+$', expand=True),
    df_id['filename'].str.extract(r'/scenario(?P<i_map>\d+)-(?P<i_locations>\d+).json$', expand=True).astype(int),
    df_id['string_probabilityForcingForHuman'].str.extract(r'^probabilityForcingForHuman (?P<probabilityForcingForHuman>[\d.]+)$', expand=True).astype(float),
], axis=1).rename(columns={'i_locations': 'Positions variant'})
df_id['are_bridges'] = df_id['dir_map'] == '2024-11-20_14:24:09'
df_id

,filename,Coordination strategy,string_seed,string_probabilityForcingForHuman,heuristic,dir_map,i_map,Positions variant,probabilityForcingForHuman,are_bridges
0,map-generator/generated-maps/2024-11-20_14:24:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-20_14:24:09,1,1,0.0,True
1,map-generator/generated-maps/2024-11-20_14:24:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-20_14:24:09,1,1,1.0,True
2,map-generator/generated-maps/2024-11-20_14:24:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-20_14:24:09,1,1,1.0,True
3,map-generator/generated-maps/2024-11-20_14:24:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-20_14:24:28,1,1,0.0,False
4,map-generator/generated-maps/2024-11-20_14:24:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-20_14:24:28,1,1,1.0,False
...,...,...,...,...,...,...,...,...,...,...
295,map-generator/generated-maps/2024-11-20_14:24:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-20_14:24:09,1,10,1.0,True
296,map-generator/generated-maps/2024-11-20_14:24:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-20_14:24:09,1,10,1.0,True
297,map-generator/generated-maps/2024-11-20_14:24:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-20_14:24:28,1,10,0.0,False
298,map-generator/generated-maps/2024-11-20_14:24:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-20_14:24:28,1,10,1.0,False


In [4]:
index_id = df_orig.columns.get_loc('Scenario ID')
df = pd.concat([df_id[['are_bridges', 'Coordination strategy', 'probabilityForcingForHuman', 'Positions variant']], df_orig.iloc[:, index_id + 1:]], axis=1)
df

,are_bridges,Coordination strategy,probabilityForcingForHuman,Positions variant,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,...,No. of critical sections,No. of near-misses,No. of collisions,Is blocked,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s)
0,True,baseline,0.0,1,0,HumanDrivenVehicle,78.03,0,62.2,2,...,146,0,0,1,567.1,547.2,599.9,0.3,5.6,0.1
1,True,change of priorities,1.0,1,0,HumanDrivenVehicle,78.03,0,61.8,1,...,146,0,0,1,574.4,573.3,599.9,0.3,5.6,0.1
2,True,stops,1.0,1,0,HumanDrivenVehicle,78.03,0,61.8,1,...,146,0,0,1,574.4,573.3,599.9,0.3,5.6,0.1
3,False,baseline,0.0,1,0,HumanDrivenVehicle,78.20,1,80.3,33,...,166,0,2,0,536.0,29.7,599.9,0.3,5.6,0.1
4,False,change of priorities,1.0,1,0,HumanDrivenVehicle,78.20,12,998.7,25,...,159,1,6,0,71.9,16.2,599.9,0.3,5.6,1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,True,change of priorities,1.0,10,4,AutonomousVehicle,62.11,13,1602.3,18,...,79,0,0,0,51.8,10.2,599.9,0.3,5.6,2.7
296,True,stops,1.0,10,4,AutonomousVehicle,62.11,1,180.6,4,...,16,0,0,1,520.3,496.4,599.9,0.3,5.6,0.3
297,False,baseline,0.0,10,4,AutonomousVehicle,60.61,15,1849.8,16,...,55,0,3,0,12.6,1.4,599.9,0.3,5.6,3.1
298,False,change of priorities,1.0,10,4,AutonomousVehicle,60.61,12,1459.5,18,...,74,0,0,0,63.9,13.7,599.9,0.3,5.6,2.4


In [5]:
series_blocks = df[df['Vehicle type'] != 'HumanDrivenVehicle'].groupby(['Positions variant'])['Is blocked'].sum()
indices_locations_blocked = series_blocks[series_blocks != 0].index
indices_locations_nonblocked = series_blocks[series_blocks == 0].index
indices_locations_nonblocked

Index([1, 3, 7, 9], dtype='int64', name='Positions variant')

In [6]:
def save_and_show(filename):  # to avoid inlining large image data into the notebook file
    plt.savefig(filename)
    plt.close()
    # The `random` is because of https://stackoverflow.com/a/43640705.
    IPython.display.display(IPython.display.HTML(f'<img src="{filename}?{random.random()}" alt="{filename}" />'))

In [7]:
def plot_aut(df, title):
    # Extract levels
    positions = df.index.get_level_values('Positions variant').unique()
    strategies = df.index.get_level_values('Coordination strategy').unique()
    
    # Plot configuration
    fig, axes = plt.subplots(1, len(positions), figsize=(20, 6), sharey=True, squeeze=False)
    axes: list[list[Axes]]
    bar_width = 0.4
    colors = ['tab:blue', 'red', 'yellow']
    
    # Get the global min and max values for consistent y-axis scaling
    y1_min, y1_max = df['Total distance traveled (m)'].min(), df['Total distance traveled (m)'].max()
    y2_min, y2_max = min(df['No. of collisions'].min(), df['No. of near-misses'].min()), max(df['No. of collisions'].max(), df['No. of near-misses'].max())
    
    # Iterate through each Positions variant
    for i, pos in enumerate(positions):
        ax = axes[0][i]
        df_pos = df.loc[pos]
        
        # Bar positions for each Coordination strategy
        x_positions = np.arange(len(strategies))
        
        # Plot bars for 'Total distance traveled (m)'
        bars = ax.bar(x_positions, df_pos['Total distance traveled (m)'], width=bar_width, label='Distance Traveled', color=colors[0])
        ax.set_xlabel('Coordination Strategy')
        ax.set_ylabel('Total Distance Traveled (m)', color=colors[0])
        ax.tick_params(axis='y', labelcolor=colors[0])
        ax.set_ylim(0, y1_max * 1.1)
        
        # Create a secondary axis for the points
        ax_right = ax.twinx()
        
        # Plot points for 'No. of collisions' and 'No. of near-misses'
        collisions, = ax_right.plot(x_positions, df_pos['No. of collisions'], marker='o', linestyle='', label='No. of Collisions', color=colors[1])
        near_misses, = ax_right.plot(x_positions, df_pos['No. of near-misses'], marker='o', linestyle='', label='No. of Near Misses', color=colors[2])
        ax_right.set_ylabel('Collisions (r) & Near Misses (y)', color='black')
        ax_right.tick_params(axis='y', labelcolor='black')
        ax_right.set_ylim(0, y2_max * 1.1)
        
        # Add labels, grid, and title for each section
        ax.set_xticks(x_positions)
        ax.set_xticklabels(strategies, rotation=45, ha='right')
        ax.set_title(f'Positions variant {pos}')
        ax.grid(axis='y')
        
        if i == 0:
            handles = [bars, collisions, near_misses]
            labels = ['Distance Traveled', 'No. of Collisions', 'No. of Near Misses']
        
    fig.suptitle(f'{title}:\nAutomated vehicles (summarised for AV1-AV4)', fontsize=16)
    fig.legend(handles=handles, labels=labels, loc='upper left', ncol=3)
    plt.tight_layout()    
    save_and_show(title + '.png')
    IPython.display.display(df)

    
for are_bridges in (True, False):
    df_aut = df[df['are_bridges'] == are_bridges]
    df_aut = df_aut[df_aut['Positions variant'].isin(indices_locations_nonblocked)]
    df_aut = df_aut[df_aut['Vehicle type'] == 'AutonomousVehicle']
    df_aut = df_aut.groupby(['Positions variant', 'Coordination strategy']).sum()
    df_aut = df_aut[['Total distance traveled (m)', 'No. of collisions', 'No. of near-misses']]
    plot_aut(df_aut, 'With bridges' if are_bridges else 'Without bridges')

Total distance traveled (m)  \
Positions variant Coordination strategy                                
1                 baseline                                    6261.7   
                  change of priorities                        6261.5   
                  stops                                       6228.7   
3                 baseline                                    5642.2   
                  change of priorities                        5353.6   
                  stops                                       4919.9   
7                 baseline                                    5522.8   
                  change of priorities                        4736.7   
                  stops                                       4116.9   
9                 baseline                                    4714.0   
                  change of priorities                        4470.7   
                  stops                                       4123.6   

                                         No. of collisions  No. of near-misses  
Positions variant Coordination strategy                                         
1                 baseline                               0                   0  
                  change of priorities                   0                   0  
                  stops                                  0                   0  
3                 baseline                               0                   0  
                  change of priorities                   0                   0  
                  stops                                  2                   2  
7                 baseline                               2                   0  
                  change of priorities                   6                   2  
                  stops                                  8                   5  
9                 baseline                               0                   0  
                  change of priorities                   0                   0  
                  stops                                 11                   5

Total distance traveled (m)  \
Positions variant Coordination strategy                                
1                 baseline                                    6168.5   
                  change of priorities                        5324.8   
                  stops                                       4152.7   
3                 baseline                                    6622.1   
                  change of priorities                        6177.8   
                  stops                                       5657.4   
7                 baseline                                    5396.6   
                  change of priorities                        4690.4   
                  stops                                       4081.4   
9                 baseline                                    5096.8   
                  change of priorities                        4278.4   
                  stops                                       3890.0   

                                         No. of collisions  No. of near-misses  
Positions variant Coordination strategy                                         
1                 baseline                               2                   0  
                  change of priorities                   8                   1  
                  stops                                 11                   1  
3                 baseline                               7                   4  
                  change of priorities                   0                   0  
                  stops                                  6                   1  
7                 baseline                               6                   0  
                  change of priorities                   4                   1  
                  stops                                 12                   1  
9                 baseline                               8                   0  
                  change of priorities                   5                   0  
                  stops                                 16                   2

In [8]:
def plot_hum(df, title):
    # Extract levels
    positions = df.index.get_level_values('Positions variant').unique()
    strategies = df.index.get_level_values('Coordination strategy').unique()
    
    # Plot configuration
    fig, axes = plt.subplots(1, len(positions), figsize=(20, 6), sharey=True, squeeze=False)
    axes: list[list[Axes]]
    bar_width = 0.4
    colors = ['tab:blue', 'black', 'red', 'yellow']
    
    # Get the global min and max values for consistent y-axis scaling
    y1_min, y1_max = df['Total distance traveled (m)'].min(), df['Total distance traveled (m)'].max()
    # y2_min, y2_max = df['No. of forcing events'].min(), df['No. of forcing events'].max()
    y2_min, y2_max = min(df['No. of forcing events'].min(), df['No. of collisions'].min(), df['No. of near-misses'].min()), max(df['No. of forcing events'].max(), df['No. of collisions'].max(), df['No. of near-misses'].max())
    
    # Iterate through each Positions variant
    for i, pos in enumerate(positions):
        ax = axes[0][i]
        df_pos = df.loc[pos]
        
        # Bar positions for each Coordination strategy
        x_positions = np.arange(len(strategies))
        
        # Plot bars for 'Total distance traveled (m)'
        bars = ax.bar(x_positions, df_pos['Total distance traveled (m)'], width=bar_width, label='Distance Traveled', color=colors[0])
        ax.set_xlabel('Coordination Strategy')
        ax.set_ylabel('Total Distance Traveled (m)', color=colors[0])
        ax.tick_params(axis='y', labelcolor=colors[0])
        ax.set_ylim(0, y1_max * 1.1)
        
        # Create a secondary axis for the points
        ax_right = ax.twinx()
        
        # Plot points for 'No. of collisions' and 'No. of near-misses'
        forcing_events, = ax_right.plot(x_positions, df_pos['No. of forcing events'], marker='o', linestyle='', label='No. of forcing events', color=colors[1])
        collisions, = ax_right.plot(x_positions, df_pos['No. of collisions'], marker='o', linestyle='', label='No. of Collisions', color=colors[2])
        near_misses, = ax_right.plot(x_positions, df_pos['No. of near-misses'], marker='o', linestyle='', label='No. of Near Misses', color=colors[3])
        ax_right.set_ylabel('forcing events (b), collisions (r) & near misses (y)', color='black')
        # ax_right.set_ylabel('Collisions & Near Misses', color='black')
    
        ax_right.tick_params(axis='y', labelcolor='black')
        ax_right.set_ylim(0, y2_max * 1.1)
        
        # Add labels, grid, and title for each section
        ax.set_xticks(x_positions)
        ax.set_xticklabels(strategies, rotation=45, ha='right')
        ax.set_title(f'Positions variant {pos}')
        ax.grid(axis='y')
        
        if i == 0:
            handles = [bars, forcing_events, collisions, near_misses]
            labels = ['Distance Traveled', 'No. of forcing events', 'No. of Collisions', 'No. of Near Misses']
        
    fig.suptitle(f'{title}:\nHuman-driven vehicle', fontsize=16)
    fig.legend(handles=handles, labels=labels, loc='upper left', ncol=3)
    plt.tight_layout()    
    save_and_show(title + '.png')
    IPython.display.display(df)


for are_bridges in (True, False):
    df_hum = df[df['are_bridges'] == are_bridges]
    df_hum = df_hum[df_hum['Positions variant'].isin(indices_locations_nonblocked)]
    df_hum = df_hum[df_hum['Vehicle type'] == 'HumanDrivenVehicle']
    df_hum = df_hum.groupby(['Positions variant', 'Coordination strategy']).sum()
    df_hum = df_hum[['Total distance traveled (m)', 'No. of forcing events', 'No. of collisions', 'No. of near-misses']]
    plot_hum(df_hum, 'With bridges' if are_bridges else 'Without bridges')

Total distance traveled (m)  \
Positions variant Coordination strategy                                
1                 baseline                                      62.2   
                  change of priorities                          61.8   
                  stops                                         61.8   
3                 baseline                                      29.8   
                  change of priorities                        1196.2   
                  stops                                       1090.2   
7                 baseline                                      62.6   
                  change of priorities                        1088.1   
                  stops                                       1318.6   
9                 baseline                                     620.7   
                  change of priorities                        1109.1   
                  stops                                       1452.7   

                                         No. of forcing events  \
Positions variant Coordination strategy                          
1                 baseline                                   0   
                  change of priorities                       1   
                  stops                                      1   
3                 baseline                                   0   
                  change of priorities                      23   
                  stops                                     23   
7                 baseline                                   0   
                  change of priorities                      25   
                  stops                                     29   
9                 baseline                                   0   
                  change of priorities                      23   
                  stops                                     22   

                                         No. of collisions  No. of near-misses  
Positions variant Coordination strategy                                         
1                 baseline                               0                   0  
                  change of priorities                   0                   0  
                  stops                                  0                   0  
3                 baseline                               0                   0  
                  change of priorities                   0                   0  
                  stops                                  2                   2  
7                 baseline                               0                   0  
                  change of priorities                   0                   0  
                  stops                                  8                   5  
9                 baseline                               0                   0  
                  change of priorities                   0                   0  
                  stops                                 11                   5

Total distance traveled (m)  \
Positions variant Coordination strategy                                
1                 baseline                                      80.3   
                  change of priorities                         998.7   
                  stops                                       1241.4   
3                 baseline                                     619.7   
                  change of priorities                        1268.5   
                  stops                                       1164.3   
7                 baseline                                     429.7   
                  change of priorities                         998.9   
                  stops                                       1169.7   
9                 baseline                                     604.1   
                  change of priorities                         984.9   
                  stops                                       1402.0   

                                         No. of forcing events  \
Positions variant Coordination strategy                          
1                 baseline                                   0   
                  change of priorities                      21   
                  stops                                     27   
3                 baseline                                   0   
                  change of priorities                      24   
                  stops                                     20   
7                 baseline                                   0   
                  change of priorities                      23   
                  stops                                     27   
9                 baseline                                   0   
                  change of priorities                      18   
                  stops                                     20   

                                         No. of collisions  No. of near-misses  
Positions variant Coordination strategy                                         
1                 baseline                               2                   0  
                  change of priorities                   6                   1  
                  stops                                 11                   1  
3                 baseline                               7                   2  
                  change of priorities                   0                   0  
                  stops                                  6                   1  
7                 baseline                               6                   0  
                  change of priorities                   2                   1  
                  stops                                 12                   1  
9                 baseline                               6                   0  
                  change of priorities                   5                   0  
                  stops                                 16                   2

In [9]:
def show_maps(title, indices, ncols): 
    nrows = max(1, (len(indices) + ncols - 1) // ncols)
    fig, axes_matrix = plt.subplots(nrows, ncols, figsize=(16, 9), squeeze=False)
    #print(f'{title}: {nrows}x{ncols}')

    axes = list(itertools.chain.from_iterable(axes_matrix))
    axes_matrix: list[list[Axes]]
    assert len(axes) >= len(indices)
    
    for ax in axes:
        ax.axis('off')
        
    for ax, i_pos in zip(axes, indices):
        image = plt.imread(f'../screenshotting/screenshots/cropped/{i_pos}.png')
        ax.imshow(image)
        ax.title.set_text(f'Positions variant {i_pos}')
    
    fig.suptitle(title, fontsize=16)

    plt.subplots_adjust(wspace=0.1, hspace=0.3)
    plt.tight_layout()
    save_and_show(title + '.png')
    
show_maps('Non-blocked', indices_locations_nonblocked, 4)
show_maps('Blocked', indices_locations_blocked, 4)